In [17]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile, assemble, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram
from math import gcd
from numpy.random import randint
import pandas as pd
from fractions import Fraction

In [18]:
def generateQTCC(CuT, CuT_output_indexes, input, result):

  	# Create the registers..
	# We need:
	#   -The Input register, with as many qubits as the CuT's input, 
	#   -The Expected register, with as many qubits as significative output qubits of the CuT,
    #   -The Check register, with the same amount of qubits as the previous one,
	#   -One qubit for the verdict.
	QTCC=QuantumCircuit()
	CuT_registers = []
	for register in CuT.qregs:
		for qubit in register: 
			CuT_registers.append(qubit)
	CuT_qr = QuantumRegister(len(CuT_registers),'cut_input')
	expectedValue_qr = QuantumRegister(len(CuT_output_indexes),'expected')
	valueCheck_qr = QuantumRegister(len(CuT_output_indexes),'check')
	verdict_qr = QuantumRegister(1,'q_verdict')
	verdict_bit = ClassicalRegister(1,'c_verdict')
	QTCC.add_register(CuT_qr)    
	QTCC.add_register(expectedValue_qr)
	QTCC.add_register(valueCheck_qr)
	QTCC.add_register(verdict_qr)
	QTCC.add_register(verdict_bit)

	# Inicializamos los qubits de entrada y los resultados esperados
	i = 0
	for value in input:
		if value == 1:
			QTCC.x(CuT_qr[i])
		i = i+1
	i = 0
	for value in result:
		if value == 1:
			QTCC.x(expectedValue_qr[i])
		i = i+1
	QTCC.barrier(CuT_qr, expectedValue_qr, valueCheck_qr, verdict_qr)
    # We add the Circuit under Testing
	QTCC.append(CuT, CuT_qr)
	QTCC.barrier(CuT_qr, expectedValue_qr, valueCheck_qr, verdict_qr)
    # Añadimos las puertas que hacen la verificación
	n_qubit = 0
	for index in CuT_output_indexes:
		QTCC.ccx(CuT_qr[index], expectedValue_qr[n_qubit], valueCheck_qr[n_qubit])
		QTCC.ccx(CuT_qr[index], expectedValue_qr[n_qubit], valueCheck_qr[n_qubit], ctrl_state='00')
		n_qubit = n_qubit+1
	QTCC.barrier(CuT_qr, expectedValue_qr, valueCheck_qr, verdict_qr)
    # Volcamos el resultado de la prueba en un bit clásico 
	QTCC.mcx(valueCheck_qr, verdict_qr)
	QTCC.measure(verdict_qr, verdict_bit)

	return QTCC

def executeQTCC(circuit, shots):
    #Creamos el simulador
    simulator = Aer.get_backend('aer_simulator') 
    # Transpile for simulator
    circuit = transpile(circuit, simulator)
    # Do the simulation and return the result
    result = simulator.run(circuit, shots = shots, memory=True).result()
    counts = result.get_counts()
    memory = result.get_memory(circuit)
    # Comprobamos el bit de veredicto
    verdict = False
    # Si el qubit de veredicto ha colapasado a 0 al menos una vez, 
    if '1' in counts.keys():
        verdict = True        

    return verdict

In [19]:
def test_process(CuT, cut_output_indexes, test_suite, shots):

    # Comprobar que el tamaño de los vectores de prueba se corresponde 
    # con el número de entradas y salidas del circuito
    correct_format = True
    for test_case in test_suite:
        if len(cut_output_indexes) != len(test_case[1]):
            correct_format = False
    if correct_format == False:
        return 'Wrong inputs: Input/output vectors lenght inconsistent with circuit size'       

    # Creamos un set para almacenar los vectores 
    # que no han superado la prueba
    failed_tests = []
    # Para cada vector de prueba, ejecutamos el test y almacenamos
    # el vector en caso de fallo
    for test_case in test_suite:
        input = test_case[0]
        expected_result = test_case[1]
        verdict = True
        # Lo ejecutamos
        QTCC = generateQTCC(CuT, cut_output_indexes, input, expected_result)
        verdict = executeQTCC(QTCC, shots)
        # Si la prueba ha fallado, añadimos el caso de prueba a el vector de fallos
        if verdict == False:
            failed_tests.append(test_case)
    # Mostramos el resultado de las pruebas
    if len(failed_tests) == 0:
        msg = 'Test suite passed: all tests successful'
    else:
        msg ='Test unsuccessful. Failed tests: '
        msg = msg + str(len(failed_tests)) + '/' + str(len(test_suite))+ " "
        msg = msg + str(failed_tests)

    return msg, QTCC

In [20]:
def generateAdder():   
    adder = QuantumCircuit()
    input_qr = QuantumRegister(4,'input')
    adder.add_register(input_qr)
    output_indexes = (2, 3)

    adder.ccx(input_qr[0], input_qr[1], input_qr[3])
    adder.cx(input_qr[0], input_qr[1])
    adder.ccx(input_qr[1], input_qr[2], input_qr[3])
    adder.cx(input_qr[1], input_qr[2])
    adder.cx(input_qr[0], input_qr[1])
    adder.name = "adder"

    test_suite = (((0,0,0), (0,0)),
                  ((0,0,1), (1,0)),
                  ((0,1,0), (1,0)),
                  ((0,1,1), (0,1)),
                  ((1,0,0), (1,0)),
                  ((1,0,1), (0,1)),
                  ((1,1,0), (0,1)),
                  ((1,1,1), (1,1)))
    
    return (adder, output_indexes, test_suite)

def generateAdder_Bugged():   
    testData = generateAdder()
    CuT = testData[0]
    outputIndexes = testData[1]
    CuT.name = "adder_bugged"

    test_suite = (((0,0,0), (1,0)), #Error
                  ((0,0,1), (1,0)), 
                  ((0,1,0), (1,1)), #Error
                  ((0,1,1), (0,1)),
                  ((1,0,0), (0,0)), #Error
                  ((1,0,1), (0,1)),
                  ((1,1,0), (0,1)),
                  ((1,1,1), (1,1)))
    
    return (CuT, outputIndexes, test_suite)

In [21]:
def c_amod15(a, power):
    """Controlled multiplication by a mod 15"""
    if a not in [2,4,7,8,11,13]:
        raise ValueError("'a' must be 2,4,7,8,11 or 13")
    U = QuantumCircuit(4)        
    for iteration in range(power):
        if a in [2,13]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a in [7,8]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [4, 11]:
            U.swap(1,3)
            U.swap(0,2)
        if a in [7,11,13]:
            for q in range(4):
                U.x(q)
    print(U)
    U = U.to_gate()
    c_U = U.control()
    return c_U

def generateUnitary():   

    # Specify variables
    n_count = 8  # number of counting qubits
    a = 7
    power = 1

    QG = c_amod15(a, power)
    U_amod = QuantumCircuit()
    input_qr = QuantumRegister(5,'input')
    U_amod.add_register(input_qr)
    U_amod.append(QG, input_qr)
    U_amod.draw()
    U_amod.name = "%i^%i mod 15" % (a, power)
    output_indexes = (0,1,2,3,4)
    
    test_suite = (
                  ((0,0,0,1,0), (0,0,0,1,0)),
                  ((0,0,0,1,1), (0,0,0,1,1)),
                  ((1,0,0,1,0), (1,1,0,1,1)),
                  ((1,0,0,1,1), (1,1,0,0,1)))
    
    return (U_amod, output_indexes, test_suite)

def generateUnitary_Bugged():   
    testData = generateCmod()
    CuT = testData[0]
    outputIndexes = testData[1]
    CuT.name = CuT.name+"_bugged"
    test_suite = (((0,0,0,1,0), (0,0,1,1,1)),#Error
                  ((0,0,0,1,1), (1,0,0,1,1)),#Error
                  ((1,0,0,1,0), (1,1,0,1,1)),
                  ((1,0,0,1,1), (1,1,0,0,0)))#Error
    
    return (CuT, outputIndexes, test_suite)

In [22]:
#SELECT CUT 

#Bell preparator
# generator = generateBell

#Bugged Bell preparator
#generator = generateBell_Bugged

#Full Adder
#generator = generateAdder

#Bugged Full Adder
#generator = generateAdder_Bugged

#Shor's unitary
generator = generateUnitary

#Bugged Shor's unitary
#generator = generateUnitary_Bugged

In [23]:
testData = generator()
CuT = testData[0]
outputIndexes = testData[1]
testSuite = testData[2]
msg, QTCC = test_process(CuT, outputIndexes, testSuite, 100)
print(QTCC)
print('CUT: ' + CuT.name)
print('test suite:')
print(testSuite)
print(msg)

        ┌───┐          
q_0: ─X─┤ X ├──────────
      │ └───┘┌───┐     
q_1: ─X───X──┤ X ├─────
          │  └───┘┌───┐
q_2: ─────X────X──┤ X ├
               │  ├───┤
q_3: ──────────X──┤ X ├
                  └───┘
             ┌───┐ ░ ┌─────────────┐ ░                                         »
cut_input_0: ┤ X ├─░─┤0            ├─░───■────────────────────────o────────────»
             └───┘ ░ │             │ ░   │                        │            »
cut_input_1: ──────░─┤1            ├─░───┼────■───────────────────┼────o───────»
                   ░ │             │ ░   │    │                   │    │       »
cut_input_2: ──────░─┤2 7^1 mod 15 ├─░───┼────┼────■──────────────┼────┼────o──»
             ┌───┐ ░ │             │ ░   │    │    │              │    │    │  »
cut_input_3: ┤ X ├─░─┤3            ├─░───┼────┼────┼────■─────────┼────┼────┼──»
             ├───┤ ░ │             │ ░   │    │    │    │         │    │    │  »
cut_input_4: ┤ X ├─░─┤4            ├─░───┼────┼────┼───